In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime as dt
%matplotlib inline

import requests
import json
import calendar

In [2]:
tweets_df = pd.read_pickle('./XRPDataSet.pkl')

In [3]:
tweets_df['datetime'] = tweets_df['datetime'].apply(lambda x : x.to_pydatetime())

In [4]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25435 entries, 0 to 25434
Data columns (total 13 columns):
ID               25435 non-null float64
datetime         25435 non-null datetime64[ns]
is_reply         25435 non-null float64
is_retweet       25435 non-null float64
nbr_favorite     25435 non-null float64
nbr_reply        25435 non-null float64
nbr_retweet      25435 non-null float64
text             25435 non-null object
url              25435 non-null object
user_id          25435 non-null object
usernameTweet    25435 non-null object
has_media        8834 non-null float64
medias           8834 non-null object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 2.5+ MB


In [5]:
tweets_df.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias
0,9.885302e+17,2018-04-23 17:29:08,0.0,0.0,0.0,0.0,0.0,New post (Could Ripple XRP Really Reach $1...,/ExcelVisibility/status/988530235493748736,815322320,ExcelVisibility,NaN,NaN
1,9.847927e+17,2018-04-13 09:57:41,0.0,0.0,0.0,0.0,0.0,"Did you know that you can buy, sell, store, co...",/coindirectcom/status/984792745666580481,915453520924545025,coindirectcom,1.0,[https://t.co/h1CTJHGwPs]
2,9.863800e+17,2018-04-17 19:05:02,0.0,0.0,0.0,0.0,0.0,1 Ripple = 0.6589 USD. Ripple has changed ...,/RippleMarket/status/986380040517701633,848989770923601922,RippleMarket,NaN,NaN
3,9.853410e+17,2018-04-14 22:16:19,0.0,0.0,0.0,0.0,1.0,# Batteries #ITO #ICO #Token #TokenSale #...,/nsdelpitiya/status/985341014339608576,880003387672088576,nsdelpitiya,NaN,NaN
4,9.866047e+17,2018-04-18 09:57:42,0.0,0.0,0.0,0.0,0.0,How Is #Ripple Different From All Other #Cryp...,/Matthix191/status/986604686856933376,916641996823031808,Matthix191,1.0,[https://t.co/xF5l49lz8P]


In [6]:
min(tweets_df['datetime'])

Timestamp('2018-04-13 07:09:42')

In [7]:
def cielDatetime(d):
    delta = 3600-((d.minute*60+d.second)%3600)
    
    td = dt.timedelta(seconds=delta)
    return d.to_pydatetime()+td

def datetimeToZulu(d):
    td = dt.timedelta(minutes=240)
    return d.to_pydatetime()+td

In [8]:
tweets_df['rounded_dateTime'] = tweets_df['datetime'].apply(lambda t : cielDatetime(t))
# tweets_df['rounded_zulu'] = tweets_df['rounded_dateTime'].apply(lambda t : datetimeToZulu(t))

In [9]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25435 entries, 0 to 25434
Data columns (total 14 columns):
ID                  25435 non-null float64
datetime            25435 non-null datetime64[ns]
is_reply            25435 non-null float64
is_retweet          25435 non-null float64
nbr_favorite        25435 non-null float64
nbr_reply           25435 non-null float64
nbr_retweet         25435 non-null float64
text                25435 non-null object
url                 25435 non-null object
user_id             25435 non-null object
usernameTweet       25435 non-null object
has_media           8834 non-null float64
medias              8834 non-null object
rounded_dateTime    25435 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(7), object(5)
memory usage: 2.7+ MB


In [10]:
dateTimeGroup = tweets_df.groupby('rounded_dateTime')['ID'].count().rename('tweetCount').reset_index()

In [11]:
dateTimeGroup.head(10)

,rounded_dateTime,tweetCount
0,2018-04-13 08:00:00,83
1,2018-04-13 09:00:00,131
2,2018-04-13 10:00:00,169
3,2018-04-13 11:00:00,116
4,2018-04-13 12:00:00,112
5,2018-04-13 13:00:00,123
6,2018-04-13 14:00:00,135
7,2018-04-13 15:00:00,93
8,2018-04-13 16:00:00,110
9,2018-04-13 17:00:00,117


In [12]:
dateTimeGroup.describe()

,tweetCount
count,267.000000
mean,95.262172
std,28.270687
min,42.000000
25%,73.500000
50%,94.000000
75%,111.000000
max,221.000000


In [13]:
url = 'https://min-api.cryptocompare.com/data/histohour?fsym=XRP&tsym=USD&limit=297&aggregate=1&e=CCCAGG'
response = requests.get(url)
historical_xrp_prices = json.loads(response.text)
fromDate = historical_xrp_prices['TimeFrom']
toDate = historical_xrp_prices['TimeTo']

In [14]:
print(str(len(historical_xrp_prices['Data'])) + ' Price Samples')
print('From: ' + str(dt.datetime.utcfromtimestamp(int(fromDate))))
print('To: ' + str(dt.datetime.utcfromtimestamp(int(toDate)))) 

298 Price Samples
From: 2018-04-14 03:00:00
To: 2018-04-26 12:00:00


In [15]:
priceData = historical_xrp_prices['Data']

xrp_prices = pd.DataFrame()

for price in priceData:
    price['time'] = dt.datetime.fromtimestamp(price['time'])
    xrp_prices = xrp_prices.append(price, ignore_index=True)

NameError: name 'priceData' is not defined

In [ ]:
xrp_prices.tail()

In [ ]:
xrp_prices.to_pickle('./xrp_price_data_hourly.pkl')

In [ ]:
sns.distplot(dateTimeGroup['tweetCount'])

In [17]:
dateTimeGroup.to_pickle('./xrp_grouped_by_time')

In [ ]:
tweets_df.to_pickle('./xrp_wi')